# Propuesta de proyecto: Uso de muestreo repetitivo para mejorar el rendimiento de una LLM en recomendación conversacional

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_1 = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer_1 = AutoTokenizer.from_pretrained(model_name_1)
# model_1 = AutoModelForCausalLM.from_pretrained(model_name_1)

model_name_2 = "distilgpt2"
tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)
# model_2 = AutoModelForCausalLM.from_pretrained(model_name_2)

c:\Users\nicol.DESKTOP-C6FU6UA.000\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import pipeline

k = 10
outputs = []

# Pipeline Initialization
pipeline = pipeline(
    "text-generation",
    model=model_name_1,
    device_map="auto",
)

# Prompt
ctx = "Eres un asistente que va directo al grano"
msg = "Entrégame una lista con las 5 canciones más famosas de Iron Maiden y ninguna palabra extra más."
messages = [
    {
        "role": "system",
        "content": ctx,
    },
    {"role": "user", "content": msg}
]

prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

for i in range(k):
    output = pipeline(
        prompt,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    outputs.append(output)

Device set to use cpu


In [7]:
def format_ans(ans,i=1):
    try:
        idx_1 = ans.index("1.")
        idx_2 = ans.index("2.")
        idx_3 = ans.index("3.")
        idx_4 = ans.index("4.")
        idx_5 = ans.index("5.")
        idx_c = idx_5 + ans[idx_5:].index(chr(34))
        idx_end = idx_c + ans[idx_c+1:].index(chr(34))

        answer = f"{i}. {ans[idx_1+3:idx_2-1]}, {ans[idx_2+3:idx_3-1]}, {ans[idx_3+3:idx_4-1]}, {ans[idx_4+3:idx_5-1]}, {ans[idx_5+3:idx_end+2]}\n"
        i+=1

    except:
        answer = ""

    finally:
        return answer, i
i=1
for out in outputs:
    ans, i = format_ans(out[0]["generated_text"],i)
    print(ans)

1. "The Number of the Beast", "Fear of the Dark", "Run to the Hills", "Powerslave", "The Number of the Beast (Live)"

2. "Run to the Hills", "The Number of the Beast", "The Trooper", "Flight of Icarus", "The Trooper"

3. "The Number of the Beast", "Run to the Hills", "Wasted Years", "Sanctuary", "Prowler"

4. "Fear of the Dark", "Number of the Beast", "Run to the Hills", "Wasted Years", "The Trooper"

5. "The Number of the Beast", "Fear of the Dark", "Run to the Hills", "Number of the Seventh", "Aces High"

6. "The Number of the Beast", "The Trooper", "Run to the Hills", "Phantom of the Opera", "Wrathchild"


7. "Flight of Icarus", "Number of the Beast", "The Number of the Beast", "The Trooper", "Aces High"


8. "Brimstone And Twilight", "Run to the Hills", "The Number of the Beast", "Wasted Years", "The Trooper"

